In [155]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
%matplotlib inline
import pandas as pd

In [156]:
voltage = 400 #nominal_voltage
battery_volumn = 60 * 1000 / 400 # Q = kWh / v
emission_max_value = 100
# start_soc = 0.216
# end_soc = 0.7
# target_charge_volumn = (end_soc - start_soc) *battery_volumn
Power = 10 * 1000 # power of the charger
I_max = Power  / voltage
R = 0.001 # resistance
Power_limit = 10 * 1000 # simple assumption to the limit of the power: 100 kw
Power_limit_slope_line_Intercept = 50 * 1000 # simple assumption to the limit of the power(the sloped line): 100 kw

In [157]:
step = 1/12 # 10(min)/ 60(min)

In [158]:
print(step)

0.08333333333333333


In [159]:
print(24 / step)

288.0


In [160]:
maximum_steps = 24 / step # 24 hours divided by step

In [161]:
x = np.linspace(0, int(maximum_steps), int(maximum_steps)+1)
emission_array = emission_max_value/((maximum_steps/2)**2) * (x-(maximum_steps/2))**2
emission_array = np.concatenate((emission_array[:-1], emission_array[:-1]), axis=0)


In [162]:
emission_array = emission_array.reshape(len(emission_array),1)
print(emission_array.shape)

(576, 1)


In [163]:
maximum_steps = int(2* maximum_steps)

In [164]:
print(maximum_steps)

576


In [165]:
num_vehicles = 2 

In [166]:
# basic = {
#          0:{"start_time":250,
#              "end_time":350,
#              "start_soc":0.2,
#              "end_soc":0.7}
#              }

In [167]:
basic = {0:{"start_time":144,
             "end_time":144+287,
             "start_soc":0.2,
             "end_soc":0.7},
         1:{"start_time":260,
             "end_time":320,
             "start_soc":0.4,
             "end_soc":0.7}
             }

In [168]:
target_charge_volumn = []
for i in range(num_vehicles):
    target_charge_volumn.append((basic[i]["end_soc"]-basic[i]["start_soc"])*battery_volumn)

In [169]:
current_state = cp.Variable((num_vehicles,maximum_steps), 'current at each step')
P = cp.Variable((num_vehicles,maximum_steps), 'power of the charger at each step')
soc = cp.Variable((num_vehicles,maximum_steps), "state of charge")
voltage = cp.Variable((num_vehicles,maximum_steps), "voltage")

In [170]:
P_sum = cp.sum(P, axis=0)
P_sum_pos = cp.pos(P_sum)
P_sum_reshaped = cp.reshape(P_sum_pos, (maximum_steps, 1))
objective = cp.Minimize(cp.sum(cp.multiply(P_sum_reshaped, emission_array)))
constraints = []

In [171]:
# discharge_discount_factor = 1
# positive_part = cp.pos(P)
# negative_part = cp.neg(P)
# discounted_negative_part = discharge_discount_factor  * negative_part

# P_sum = cp.sum(positive_part+ discounted_negative_part, axis=0)
# P_sum_pos = cp.pos(P_sum)
# P_sum_reshaped = cp.reshape(P_sum_pos, (maximum_steps, 1))

# objective = cp.Minimize(cp.sum(cp.multiply(P_sum_reshaped, emission_array)))
# constraints = []

In [172]:
for i in range(num_vehicles):
    for j in range(0, maximum_steps):
        constraints += [voltage[i][j] == 400]

In [173]:
for i in range(num_vehicles):
    for j in range(0, maximum_steps):
        constraints += [P[i][j] == 2 * R * current_state[i][j] + 400 * current_state[i][j]]

In [174]:
for i in range(num_vehicles):
    for j in range(basic[i]['start_time']):
        constraints += [current_state[i][j] == 0]
        constraints += [soc[i][j] == 0]

In [175]:
for i in range(num_vehicles):
    for j in range(basic[i]['end_time'],maximum_steps):
        constraints += [current_state[i][j] == 0]

In [176]:
for i in range(num_vehicles):
    constraints += [soc[i][basic[i]['start_time']] == basic[i]['start_soc']]

In [177]:
for i in range(num_vehicles):
    for j in range(basic[i]['start_time'] + 1, basic[i]['end_time'] + 1):
        constraints += [soc[i][j] == soc[i][j-1] + current_state[i][j-1] * step / battery_volumn]

In [178]:
for i in range(num_vehicles):
    for j in range(maximum_steps):
        constraints += [P[i][j] <= Power_limit_slope_line_Intercept * (1 - soc[i][j])]

In [179]:
for i in range(num_vehicles):
    constraints += [cp.sum(current_state[i]) * step >= (target_charge_volumn[i])]
constraints += [P <= Power]
constraints += [P >= -Power]

In [180]:
problem = cp.Problem(objective, constraints)
emission_volume = problem.solve()

In [181]:
emission_volume

808875.4923122934

In [182]:
P_values = P.value

In [129]:
for i in range(num_vehicles):
    for j in range(maximum_steps):
        if abs(P_values[i][j]) > 1:
            print(i," ",j," ",P_values[i][j])

0   144   9999.99999997271
0   145   9999.99999997265
0   146   9999.999999972539
0   147   9999.999999972355
0   148   9999.999999972082
0   149   9999.999999971717
0   150   9999.999999971258
0   151   9999.999999970698
0   152   9999.999999970027
0   153   9999.999999969234
0   154   9999.999999968306
0   155   9999.999999967225
0   156   9999.99999996597
0   157   9999.999999964513
0   158   9999.999999962813
0   159   9999.999999960832
0   160   9999.999999958507
0   161   9999.999999955757
0   162   9999.999999952477
0   163   9999.999999948519
0   164   9999.999999943673
0   165   9999.999999937627
0   166   9999.99999992991
0   167   9999.999999919753
0   168   9999.99999990584
0   169   9999.999999885702
0   170   9999.999999854143
0   171   9999.999999798285
0   172   9999.99999967989
0   173   6884.095011169875
0   260   -3599.9857836416086
0   261   -3585.9617542977985
0   262   -3572.853131289066
0   263   -3560.6491584212886
0   264   -3549.3387269484288
0   265   -3538.9

In [ ]:
print(P.value)

In [ ]:
print(sum(current_state[0].value))

In [ ]:
target_charge_volumn[0]